In [1]:
import pickle
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action = 'ignore')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [3]:
d = pickle.load(open("data_sjj.pkl","rb"))
d

,label,age_range,gender,action_type_0,action_type_1,action_type_2,action_type_3,action_sum,action_ratio,type_0_ratio,...,us_trans_ratio_cart,us_trans_ratio_favor,user_all_item,all_item_unique,all_cat_unique,all_seller_unique,all_brand_unique,all_time_unique,all_action_unique,time_active
0,0,6.0,0.0,410.0,0.0,34.0,7.0,451.0,-11.710938,0.009048,...,2.312500,-0.551270,451,256,45,109,106,47,3,590
1,0,6.0,0.0,410.0,0.0,34.0,7.0,451.0,-11.710938,0.009048,...,2.312500,2.312500,451,256,45,109,106,47,3,590
2,1,6.0,0.0,410.0,0.0,34.0,7.0,451.0,-11.710938,0.009048,...,4.097656,4.097656,451,256,45,109,106,47,3,590
3,0,6.0,0.0,410.0,0.0,34.0,7.0,451.0,-11.710938,0.009048,...,2.312500,2.312500,451,256,45,109,106,47,3,590
4,0,0.0,0.0,47.0,0.0,7.0,0.0,54.0,-13.835938,-0.030075,...,2.312500,2.312500,54,31,17,20,19,16,2,510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260859,0,4.0,1.0,107.0,0.0,9.0,1.0,117.0,-13.062500,0.014427,...,3.000000,3.000000,117,49,25,33,32,12,3,191
260860,0,0.0,1.0,162.0,0.0,5.0,31.0,198.0,-12.531250,-0.085388,...,2.312500,-0.861328,198,89,20,38,36,6,3,186
260861,0,0.0,1.0,162.0,0.0,5.0,31.0,198.0,-12.531250,-0.085388,...,2.312500,0.009048,198,89,20,38,36,6,3,186
260862,0,0.0,1.0,162.0,0.0,5.0,31.0,198.0,-12.531250,-0.085388,...,2.312500,2.312500,198,89,20,38,36,6,3,186


In [4]:
d.isnull().sum()

label                   0
age_range               0
gender                  0
action_type_0           0
action_type_1           0
action_type_2           0
action_type_3           0
action_sum              0
action_ratio            0
type_0_ratio            0
type_1_ratio            0
type_2_ratio            0
type_3_ratio            0
trans_ratio_ckick       0
trans_ratio_cart        0
trans_ratio_favor       0
seller_id               0
us_action_type_0        0
us_action_type_1        0
us_action_type_2        0
us_action_type_3        0
us_action_sum           0
us_action_ratio         0
us_type_0_ratio         0
us_type_1_ratio         0
us_type_2_ratio         0
us_type_3_ratio         0
us_trans_ratio_ckick    0
us_trans_ratio_cart     0
us_trans_ratio_favor    0
user_all_item           0
all_item_unique         0
all_cat_unique          0
all_seller_unique       0
all_brand_unique        0
all_time_unique         0
all_action_unique       0
time_active             0
dtype: int64

In [5]:
d.dtypes

label                      int8
age_range               float16
gender                  float16
action_type_0           float16
action_type_1           float16
action_type_2           float16
action_type_3           float16
action_sum              float16
action_ratio            float16
type_0_ratio            float16
type_1_ratio            float16
type_2_ratio            float16
type_3_ratio            float16
trans_ratio_ckick       float16
trans_ratio_cart        float16
trans_ratio_favor       float16
seller_id                 int16
us_action_type_0        float16
us_action_type_1        float16
us_action_type_2        float16
us_action_type_3        float16
us_action_sum           float16
us_action_ratio         float16
us_type_0_ratio         float16
us_type_1_ratio         float16
us_type_2_ratio         float16
us_type_3_ratio         float16
us_trans_ratio_ckick    float16
us_trans_ratio_cart     float16
us_trans_ratio_favor    float16
user_all_item             int16
all_item

In [6]:
def getb(estimator,params,x_tr,x_te,y_tr,y_te):
    grid = GridSearchCV(estimator,
                        params,
                        cv = 3,
                        refit = True)
    grid.fit(x_tr,y_tr)
    y_pre = grid.predict(x_te)
    print(classification_report(y_te,y_pre))
    print(grid.best_params_)

In [7]:
lr_p = {"C":[1e-2,1e-1,1e-3,1e-0],'penalty':['l1','l2']}
dtc_p = {"max_depth":[1,3,4,6,7,9]}
knc_p = {"n_neighbors":[3,4,5,6,7]}
lr = LogisticRegression()
dtc = DecisionTreeClassifier()
knc = KNeighborsClassifier()

In [8]:
col = [x for x in d.columns if x not in ['user_id','merchant_id','label']]
col

['age_range',
 'gender',
 'action_type_0',
 'action_type_1',
 'action_type_2',
 'action_type_3',
 'action_sum',
 'action_ratio',
 'type_0_ratio',
 'type_1_ratio',
 'type_2_ratio',
 'type_3_ratio',
 'trans_ratio_ckick',
 'trans_ratio_cart',
 'trans_ratio_favor',
 'seller_id',
 'us_action_type_0',
 'us_action_type_1',
 'us_action_type_2',
 'us_action_type_3',
 'us_action_sum',
 'us_action_ratio',
 'us_type_0_ratio',
 'us_type_1_ratio',
 'us_type_2_ratio',
 'us_type_3_ratio',
 'us_trans_ratio_ckick',
 'us_trans_ratio_cart',
 'us_trans_ratio_favor',
 'user_all_item',
 'all_item_unique',
 'all_cat_unique',
 'all_seller_unique',
 'all_brand_unique',
 'all_time_unique',
 'all_action_unique',
 'time_active']

In [9]:
x = d.loc[:,col]
x.head()

,age_range,gender,action_type_0,action_type_1,action_type_2,action_type_3,action_sum,action_ratio,type_0_ratio,type_1_ratio,...,us_trans_ratio_cart,us_trans_ratio_favor,user_all_item,all_item_unique,all_cat_unique,all_seller_unique,all_brand_unique,all_time_unique,all_action_unique,time_active
0,6.0,0.0,410.0,0.0,34.0,7.0,451.0,-11.710938,0.009048,-2.302734,...,2.312500,-0.551270,451,256,45,109,106,47,3,590
1,6.0,0.0,410.0,0.0,34.0,7.0,451.0,-11.710938,0.009048,-2.302734,...,2.312500,2.312500,451,256,45,109,106,47,3,590
2,6.0,0.0,410.0,0.0,34.0,7.0,451.0,-11.710938,0.009048,-2.302734,...,4.097656,4.097656,451,256,45,109,106,47,3,590
3,6.0,0.0,410.0,0.0,34.0,7.0,451.0,-11.710938,0.009048,-2.302734,...,2.312500,2.312500,451,256,45,109,106,47,3,590
4,0.0,0.0,47.0,0.0,7.0,0.0,54.0,-13.835938,-0.030075,-2.302734,...,2.312500,2.312500,54,31,17,20,19,16,2,510


In [72]:
y = d.loc[:,'label']
y

0         0
1         0
2         1
3         0
4         0
         ..
260859    0
260860    0
260861    0
260862    0
260863    0
Name: label, Length: 260864, dtype: int8

In [73]:
x_tr,x_te,y_tr,y_te = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [74]:
y_tr

13722     0
45886     0
227878    0
196245    0
188746    0
         ..
259178    0
103694    0
131932    1
146867    0
121958    0
Name: label, Length: 208691, dtype: int8

In [75]:
y_te

218931    0
252868    0
39752     0
209917    0
128527    1
         ..
6370      1
232460    0
215070    0
151436    1
148340    0
Name: label, Length: 52173, dtype: int8

In [76]:
# 数据分布不平衡进行 <过采样操作>
from imblearn.over_sampling import SMOTE

overstamp = SMOTE(random_state=42)

SMOTE_x_tr, SMOTE_y_tr = overstamp.fit_sample(x_tr, y_tr)
SMOTE_x_te, SMOTE_y_te = overstamp.fit_sample(x_te, y_te)
# 统计数据的标签0，1个数
print(pd.value_counts(SMOTE_y_tr, sort=True).sort_index())
print(pd.value_counts(SMOTE_y_te, sort=True).sort_index())

0    195886
1    195886
Name: label, dtype: int64
0    49026
1    49026
Name: label, dtype: int64


In [13]:
lr_p = {
    "lrc__C":[1e-2,1e-1,1e-3,1e-0],
    "lrc__penalty":["l1","l2"]}
print(lr_p)
pipe = Pipeline([('lrc',lr)])
getb(pipe,lr_p,SMOTE_x_tr,SMOTE_x_te,SMOTE_y_tr,SMOTE_y_te)

{'lrc__C': [0.01, 0.1, 0.001, 1.0], 'lrc__penalty': ['l1', 'l2']}
              precision    recall  f1-score   support

           0       0.58      0.69      0.63     49026
           1       0.62      0.50      0.55     49026

    accuracy                           0.59     98052
   macro avg       0.60      0.59      0.59     98052
weighted avg       0.60      0.59      0.59     98052

{'lrc__C': 1.0, 'lrc__penalty': 'l2'}


In [14]:
knc_params = {"classifier__n_neighbors":[3,4,5,6]}
print(knc_params)
pipeline = Pipeline([('classifier',knc)])
getb(pipeline,knc_params,SMOTE_x_tr,SMOTE_x_te,SMOTE_y_tr,SMOTE_y_te)

{'classifier__n_neighbors': [3, 4, 5, 6]}
              precision    recall  f1-score   support

           0       0.52      0.79      0.63     49026
           1       0.56      0.27      0.36     49026

    accuracy                           0.53     98052
   macro avg       0.54      0.53      0.49     98052
weighted avg       0.54      0.53      0.49     98052

{'classifier__n_neighbors': 4}


In [15]:
dtc_p = {"dtc__max_depth":[5,6,7,8,9]}
print(dtc_p)
pipeline = Pipeline([("dtc",dtc)])
getb(pipeline,dtc_p,SMOTE_x_tr,SMOTE_x_te,SMOTE_y_tr,SMOTE_y_te)

{'dtc__max_depth': [5, 6, 7, 8, 9]}
              precision    recall  f1-score   support

           0       0.84      0.99      0.91     49026
           1       0.99      0.81      0.89     49026

    accuracy                           0.90     98052
   macro avg       0.92      0.90      0.90     98052
weighted avg       0.92      0.90      0.90     98052

{'dtc__max_depth': 9}


In [16]:
gbc = GradientBoostingClassifier(learning_rate=0.1)

gbc_p = {"gbc__learning_rate":[1e-1,1e-2,1e-3]}
print(gbc_p)
pipeline = Pipeline([("gbc",gbc)])
getb(pipeline,gbc_p,SMOTE_x_tr,SMOTE_x_te,SMOTE_y_tr,SMOTE_y_te)

{'gbc__learning_rate': [0.1, 0.01, 0.001]}
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     49026
           1       1.00      0.91      0.95     49026

    accuracy                           0.95     98052
   macro avg       0.96      0.95      0.95     98052
weighted avg       0.96      0.95      0.95     98052

{'gbc__learning_rate': 0.1}


In [34]:
rfc = RandomForestClassifier(n_estimators=100)

rfc_p = {"rfc__n_estimators":[100,300]}
print(rfc_p)
pipeline = Pipeline([("rfc",rfc)])
getb(pipeline,rfc_p,SMOTE_x_tr,SMOTE_x_te,SMOTE_y_tr,SMOTE_y_te)

{'rfc__n_estimators': [100, 300]}
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     49026
           1       1.00      0.91      0.95     49026

    accuracy                           0.95     98052
   macro avg       0.96      0.95      0.95     98052
weighted avg       0.96      0.95      0.95     98052

{'rfc__n_estimators': 300}


In [38]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(SMOTE_x_tr,SMOTE_y_tr)
SMOTE_y_pre = xgb.predict(SMOTE_x_te)
print(classification_report(SMOTE_y_te,SMOTE_y_pre))

[13:01:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     49026
           1       1.00      0.93      0.96     49026

    accuracy                           0.96     98052
   macro avg       0.97      0.96      0.96     98052
weighted avg       0.97      0.96      0.96     98052



In [23]:
# 过采样 + 欠采样
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=42)
x_sm_tr, y_sm_tr = smote_enn.fit_sample(x_tr, y_tr)

print(pd.value_counts(y_sm_tr, sort=True).sort_index())

from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=42)
x_sm_tr, y_sm_tr = smote_tomek.fit_sample(x_tr, y_tr)

print(pd.value_counts(y_sm_tr, sort=True).sort_index())

0    121757
1    188229
Name: label, dtype: int64
0    194949
1    194949
Name: label, dtype: int64


In [25]:
x_sm_tr

,age_range,gender,action_type_0,action_type_1,action_type_2,action_type_3,action_sum,action_ratio,type_0_ratio,type_1_ratio,...,us_trans_ratio_cart,us_trans_ratio_favor,user_all_item,all_item_unique,all_cat_unique,all_seller_unique,all_brand_unique,all_time_unique,all_action_unique,time_active
0,0.000000,0.000000,28.000000,0.000000,2.000000,0.000000,30.0000,-14.421875,0.032776,-2.302734,...,2.312500,2.312500,30,20,10,14,16,6,2,586
1,0.000000,1.000000,226.000000,0.000000,4.000000,0.000000,230.0000,-12.382812,0.079346,-2.302734,...,2.312500,2.312500,230,123,32,25,24,22,2,491
2,0.000000,0.000000,14.000000,0.000000,3.000000,1.000000,18.0000,-14.929688,-0.130371,-2.302734,...,2.312500,2.312500,18,9,6,6,6,5,3,381
3,5.000000,0.000000,71.000000,0.000000,2.000000,0.000000,73.0000,-13.531250,0.070068,-2.302734,...,2.312500,2.312500,73,57,21,48,48,21,2,589
4,5.000000,0.000000,172.000000,0.000000,12.000000,0.000000,184.0000,-12.609375,0.034180,-2.302734,...,2.312500,2.312500,184,122,36,58,52,25,2,591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389893,0.245117,0.000000,638.000000,0.000000,25.140625,1.000000,664.5000,-11.320312,0.058807,-2.302734,...,2.943359,2.943359,664,388,59,89,89,56,3,596
389894,1.782227,0.594238,48.781250,1.811523,3.187500,2.970703,56.7500,-13.781250,-0.039398,-2.025391,...,2.720703,1.326172,56,38,15,28,29,16,3,590
389895,1.853516,0.000000,76.062500,0.000000,1.926758,0.073059,78.0625,-13.468750,0.071655,-2.302734,...,2.312500,2.312500,78,56,23,31,25,17,2,497
389896,3.597656,0.701660,87.125000,0.000000,10.578125,1.596680,99.3125,-13.226562,-0.021713,-2.302734,...,3.078125,1.418945,99,63,24,25,30,6,3,189


In [26]:
y_sm_tr

0         0
1         0
2         0
3         0
4         0
         ..
389893    1
389894    1
389895    1
389896    1
389897    1
Name: label, Length: 389898, dtype: int8

In [27]:
x_te

,age_range,gender,action_type_0,action_type_1,action_type_2,action_type_3,action_sum,action_ratio,type_0_ratio,type_1_ratio,...,us_trans_ratio_cart,us_trans_ratio_favor,user_all_item,all_item_unique,all_cat_unique,all_seller_unique,all_brand_unique,all_time_unique,all_action_unique,time_active
218931,3.0,1.0,47.0,0.0,1.0,0.0,48.0,-13.953125,0.076172,-2.302734,...,2.3125,2.312500,48,13,7,7,7,3,2,484
252868,2.0,1.0,59.0,0.0,3.0,1.0,63.0,-13.679688,0.035858,-2.302734,...,2.3125,2.312500,63,26,8,14,17,6,3,596
39752,0.0,0.0,1029.0,0.0,7.0,37.0,1073.0,-10.843750,0.057312,-2.302734,...,2.3125,-0.551270,1073,599,73,230,213,68,3,589
209917,0.0,1.0,115.0,0.0,7.0,0.0,122.0,-13.015625,0.041748,-2.302734,...,2.3125,2.312500,122,65,16,39,35,5,2,393
128527,0.0,0.0,81.0,0.0,1.0,0.0,82.0,-13.414062,0.084167,-2.302734,...,2.3125,2.312500,82,60,11,27,27,2,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6370,6.0,0.0,17.0,0.0,9.0,0.0,26.0,-14.562500,-0.282471,-2.302734,...,2.3125,2.312500,26,20,11,6,9,8,2,495
232460,3.0,0.0,57.0,0.0,10.0,0.0,67.0,-13.617188,-0.050507,-2.302734,...,2.3125,2.312500,67,32,16,15,15,6,2,404
215070,0.0,0.0,179.0,0.0,4.0,5.0,188.0,-12.585938,0.050812,-2.302734,...,2.3125,-1.216797,188,116,22,23,22,10,3,507
151436,4.0,0.0,28.0,0.0,7.0,0.0,35.0,-14.265625,-0.105347,-2.302734,...,2.3125,2.312500,35,23,13,16,16,8,2,587


In [28]:
y_te

218931    0
252868    0
39752     0
209917    0
128527    1
         ..
6370      1
232460    0
215070    0
151436    1
148340    0
Name: label, Length: 52173, dtype: int8

In [44]:
# 过采样 + 欠采样
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=42)
x_sm_tr, y_sm_tr = smote_enn.fit_sample(x_tr, y_tr)
x_sm_te, y_sm_te = smote_enn.fit_sample(x_te, y_te)

print(pd.value_counts(y_sm_tr, sort=True).sort_index())
print(pd.value_counts(y_sm_te, sort=True).sort_index())

from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=42)
x_sm_tr, y_sm_tr = smote_tomek.fit_sample(x_tr, y_tr)
x_sm_te, y_sm_te = smote_enn.fit_sample(x_te, y_te)

print(pd.value_counts(y_sm_tr, sort=True).sort_index())
print(pd.value_counts(y_sm_te, sort=True).sort_index())

0    121757
1    188229
Name: label, dtype: int64
0    30806
1    45998
Name: label, dtype: int64
0    194949
1    194949
Name: label, dtype: int64
0    30806
1    45998
Name: label, dtype: int64


In [45]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(x_sm_tr,y_sm_tr)
y_pre = xgb.predict(x_sm_te)
print(classification_report(y_sm_te,y_pre))

[13:19:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     30806
           1       1.00      0.93      0.96     45998

    accuracy                           0.96     76804
   macro avg       0.95      0.96      0.96     76804
weighted avg       0.96      0.96      0.96     76804



In [84]:
from lightgbm.sklearn import LGBMClassifier

lgb = LGBMClassifier()
lgb.fit(x_sm_tr,y_sm_tr)
y_pre = lgb.predict(x_te)
print(classification_report(y_te,y_pre))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     49026
           1       0.00      0.00      0.00      3147

    accuracy                           0.94     52173
   macro avg       0.47      0.50      0.48     52173
weighted avg       0.88      0.94      0.91     52173



In [78]:
from sklearn.metrics import roc_auc_score

In [85]:
roc_auc_score(y_te,y_pre)

0.4999490066495329

In [88]:
from lightgbm.sklearn import LGBMClassifier

lgb = LGBMClassifier()
lgb.fit(x_sm_tr,y_sm_tr)
y_pre = lgb.predict(x_sm_te)
print(classification_report(y_sm_te,y_pre))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95     30806
           1       1.00      0.93      0.96     45998

    accuracy                           0.96     76804
   macro avg       0.95      0.96      0.96     76804
weighted avg       0.96      0.96      0.96     76804



In [89]:
roc_auc_score(y_sm_te,y_pre)

0.9635311719925881